In [48]:
# Import Optuna
import optuna

# External libraries
import numpy
from tensorflow.python.keras.datasets import mnist
from tensorflow.python.keras.layers import (Dense,
                                            Flatten,
                                            Input)
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.utils import to_categorical

In [49]:
# Retrieve MNIST train/test images
((x_train, y_train), 
 (x_test, y_test)) = mnist.load_data()

# Normalize MNIST images
x_train = x_train.astype(numpy.float32) / 255 - 0.5
x_test = x_test.astype(numpy.float32) / 255 - 0.5

# Use 1-hot encoding on train and test labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Report train and test dataset properties
print('== MNIST dataset ==========================')
print(f'Train data shape   : {x_train.shape}')
print(f'Train labels shape : {y_train.shape}')
print(f'Test data shape    : {x_test.shape}')
print(f'Test labels shape  : {y_test.shape}')

== MNIST dataset ==========================
Train data shape   : (60000, 28, 28)
Train labels shape : (60000, 10)
Test data shape    : (10000, 28, 28)
Test labels shape  : (10000, 10)


In [50]:
def build_model(dense_layer_sizes,
                dense_layer_activations,
                optimizer):
    # Define input data
    input_layer = Input(shape=(28, 
                               28),
                        dtype=numpy.float32)
    
    # Define model topology
    flatten = Flatten()(input_layer)
    dense_layers = list()
    for i, (units, activation) in enumerate(zip(dense_layer_sizes,
                                                dense_layer_activations)):
        if i == 0:
            dense_layers.append(Dense(units=units,
                                      activation=activation)(flatten))
        else:
            dense_layers.append(Dense(units=units,
                                      activation=activation)(dense_layers[-1]))
    output_layer = Dense(units=10,
                         activation='sigmoid')(dense_layers[-1])
    
    # Create and compile model
    model = Model(inputs=input_layer,
                  outputs=output_layer)
    model.compile(optimizer=optimizer,
                  metrics=['accuracy'],
                  loss='categorical_crossentropy')
    
    return model

def test_model(model,
               batch_size,
               epochs,
               verbose):
    train_stats = model.fit(x=x_train,
                            y=y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            verbose=verbose).history
    (loss,
     accuracy) = model.evaluate(x=x_test,
                                y=y_test,
                                verbose=verbose)
    
    return (train_stats,
            {'loss': loss,
             'accuracy': accuracy})

## Optimize train parameters

In [51]:
def objective(trial):
    # Guess hyperparameter values
    optimizer = trial.suggest_categorical('optimizer',
                                          ['adam',
                                           'sgd'])
    batch_size = trial.suggest_int('batch_size',
                                   32,
                                   256)
    epochs = trial.suggest_int('epochs',
                               5,
                               50)
    
    # Initialize and test model
    model = build_model([16],
                        ['softmax'],
                        optimizer)
    (train_stats,
     test_stats) = test_model(model,
                              batch_size,
                              epochs,
                              False)
    
    return test_stats['accuracy']


study = optuna.create_study(direction='maximize')
study.optimize(objective,
               timeout=600)

[I 2019-09-26 16:57:38,318] Finished trial#0 resulted in value: 0.32899999618530273. Current best value is 0.32899999618530273 with parameters: {'optimizer': 'sgd', 'batch_size': 152, 'epochs': 34}.
[I 2019-09-26 16:58:23,064] Finished trial#1 resulted in value: 0.3400000035762787. Current best value is 0.3400000035762787 with parameters: {'optimizer': 'sgd', 'batch_size': 199, 'epochs': 43}.
[I 2019-09-26 16:59:14,376] Finished trial#2 resulted in value: 0.8371999859809875. Current best value is 0.8371999859809875 with parameters: {'optimizer': 'adam', 'batch_size': 251, 'epochs': 46}.
[I 2019-09-26 16:59:59,356] Finished trial#3 resulted in value: 0.22040000557899475. Current best value is 0.8371999859809875 with parameters: {'optimizer': 'adam', 'batch_size': 251, 'epochs': 46}.
[I 2019-09-26 17:01:35,004] Finished trial#4 resulted in value: 0.4927999973297119. Current best value is 0.8371999859809875 with parameters: {'optimizer': 'adam', 'batch_size': 251, 'epochs': 46}.
[I 2019-0

In [52]:
print(len(study.trials))
print(study.best_params)
print(study.best_value)

11
{'optimizer': 'adam', 'batch_size': 36, 'epochs': 36}
0.9330999851226807
